### Imports

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

POST_JDBC = "/app/postgresql-42.7.8.jar"
FILE_TABELAS_DIR = "/app/parquet_tables"
SCRATCH = "/tmp/spark_temp"

### Spark Session Builder

In [2]:
spark = SparkSession.builder \
    .appName("DataCrash-ETL") \
    .config("spark.sql.legacy.timeParserPolicy", "CORRECTED") \
    .config("spark.local.dir", SCRATCH) \
    .config("spark.master", "local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.worker.cleanup.enabled", "true") \
    .config("spark.ui.showConsoleProgress", "false") \
    \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.8") \
    \
    \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

your 131072x1 screen size is bogus. expect trouble
25/11/12 23:43:49 WARN Utils: Your hostname, DESKTOP-COU095G resolves to a loopback address: 127.0.1.1; using 172.30.167.94 instead (on interface eth0)
25/11/12 23:43:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/italo/.ivy2/cache
The jars for the packages stored in: /home/italo/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-77090f98-422f-49de-b59e-fdc9158edfdf;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.8 in central
	found org.checkerframework#checker-qual;3.49.5 in central
:: resolution report :: resolve 378ms :: artifacts dl 16ms
	:: modules in use:
	org.checkerframework#checker-qual;3.49.5 from central in [default]
	org.postgresql#postgresql;42.7.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	--------------------------------------------

### Extraction Silver Database

In [3]:
silver_path = "../Data_Layer/silver/parquet" 
df_silver = spark.read.parquet(silver_path)

df_silver.persist()

DataFrame[accident_index: string, vehicle_reference: int, longitude: double, latitude: double, accident_severity: int, number_of_vehicles: int, number_of_casualties: int, accident_timestamp: date, day_of_week: int, accident_time: string, road_type: int, speed_limit: int, junction_detail: int, junction_control: int, pedestrian_crossing_physical_facilities: int, light_conditions: int, weather_conditions: int, road_surface_conditions: int, special_conditions_at_site: int, carriageway_hazards: int, urban_or_rural_area: int, vehicle_type: int, vehicle_manoeuvre: int, vehicle_location_restricted_lane: int, was_vehicle_left_hand_drive: int, sex_of_driver: int, age_of_driver: int, age_band_of_driver: int, propulsion_code: int, age_of_vehicle: int, casualty_reference: int, casualty_class: int, sex_of_casualty: int, age_of_casualty: int, age_band_of_casualty: int, casualty_severity: int, pedestrian_movement: int, car_passenger: int, bus_or_coach_passenger: int, casualty_type: int]

### Creation of Dictionary

In [ ]:
# 1. Acidente / Ambiente
dict_severidade = {1: 'Fatal', 2: 'Sério', 3: 'Leve'}
dict_urbano_rural = {1: 'Urbano', 2: 'Rural', 3: 'Não Alocado'}
dict_dia_semana = {1: 'Domingo', 2: 'Segunda', 3: 'Terça', 4: 'Quarta', 5: 'Quinta', 6: 'Sexta', 7: 'Sábado'}
dict_clima = {
    1: 'Bom tempo', 2: 'Chuva s/ vento', 3: 'Neve s/ vento', 4: 'Ventania',
    5: 'Chuva c/ vento', 6: 'Neve c/ vento', 7: 'Neblina', 8: 'Outros', 9: 'Desc.', -1: 'Ausente'
}
dict_cod_pista = {
    1: 'Seca', 2: 'Molhada/Úmida', 3: 'Neve', 4: 'Gelo/Geada', 5: 'Alagada', 
    6: 'Óleo/Diesel', 7: 'Lama', -1: 'Ausente'
}
dict_luz = {
    1: 'Luz do Dia', 4: 'Escuro (luzes acesas)', 5: 'Escuro (luzes apagadas)', 
    6: 'Escuro (sem luzes)', 7: 'Escuro (luz desc.)', -1: 'Ausente'
}
dict_tipo_via = {
    1: 'Rotatória', 2: 'Sentido Único', 3: 'Pista Dupla', 6: 'Pista Única', 
    7: 'Via de Acesso', 9: 'Desconhecido', 12: 'Sentido Único/Acesso', -1: 'Ausente'
}
dict_cruzamento_detalhe = {
    0: 'Não em cruzamento', 1: 'Rotatória', 2: 'Mini-rotatória', 3: 'Cruzamento em T', 
    5: 'Via de acesso', 6: 'Cruzamento (4 vias)', 7: 'Mais de 4 vias', 
    8: 'Entrada privada', 9: 'Outro', -1: 'Ausente'
}
dict_cruzamento_controle = {
    0: 'Nenhum', 1: 'Pessoa Autorizada', 2: 'Semáforo', 3: 'Placa de Pare', 
    4: 'Dê a Preferência', -1: 'Ausente'
}
dict_ped_fisico = {
    0: 'Nenhuma', 1: 'Faixa de Pedestre', 4: 'Semáforo de Pedestre', 
    5: 'Fase de Pedestre no Sinal', 7: 'Passarela/Túnel', 8: 'Refúgio Central', -1: 'Ausente'
}
dict_cod_especiais = {
    0: 'Nenhuma', 1: 'Semáforo Quebrado', 2: 'Sinal Defeituoso', 3: 'Sinalização Obscura', 
    4: 'Obras na Pista', 5: 'Superfície Defeituosa', 6: 'Óleo/Diesel', 7: 'Lama', -1: 'Ausente'
}
dict_perigos_pista = {
    0: 'Nenhum', 1: 'Carga de Veículo', 2: 'Outro Objeto', 3: 'Acidente Anterior', 
    4: 'Cachorro', 5: 'Outro Animal', 6: 'Pedestre na Pista', 7: 'Animal (exceto Cavalo)', -1: 'Ausente'
}

# 2. Veículo
dict_tipo_veiculo = {
    1: 'Bicicleta', 2: 'Moto 50cc', 3: 'Moto 125cc', 4: 'Moto 125-500cc',
    5: 'Moto >500cc', 8: 'Taxi', 9: 'Carro', 10: 'Minibus', 11: 'Ônibus',
    16: 'Cavalo', 17: 'Veículo Agrícola', 18: 'Bonde (Tram)', 
    19: 'Van/Carga Leve', 20: 'Carga Média', 21: 'Carga Pesada',
    22: 'Scooter de Mobilidade', 23: 'Moto Elétrica', 90: 'Outro Veículo',
    97: 'Moto (CC Desc.)', 98: 'Carga (Peso Desc.)', -1: 'Desc.'
}
dict_manobra = {
    1: 'Ré', 2: 'Estacionado', 3: 'Parado/Aguardando', 4: 'Desacelerando', 
    5: 'Arrancando', 6: 'Retorno (U-Turn)', 7: 'Virando à Esquerda', 8: 'Aguardando Esquerda', 
    9: 'Virando à Direita', 10: 'Aguardando Direita', 11: 'Mudando Faixa (Esq)', 
    12: 'Mudando Faixa (Dir)', 13: 'Ultrapassando (movimento)', 14: 'Ultrapassando (parado)', 
    15: 'Ultrapassando (perto)', 16: 'Curva à Esquerda', 17: 'Curva à Direita', 
    18: 'Seguindo em Frente', -1: 'Ausente'
}
dict_local_veiculo = {
    0: 'Pista Principal', 1: 'Trilho de Bonde', 2: 'Faixa de Ônibus', 
    3: 'Via de Ônibus', 4: 'Ciclovia', 5: 'Calçada Compartilhada', 
    6: 'Acostamento', 7: 'Entrando no Acostamento', 8: 'Saindo do Acostamento', 
    9: 'Calçada', 10: 'Fora da Via', -1: 'Ausente'
}
dict_propulsao = {
    1: 'Gasolina', 2: 'Diesel Pesado', 3: 'Elétrico', 4: 'Vapor', 5: 'Gás', 
    6: 'GLP', 7: 'Bi-combustível', 8: 'Híbrido', 9: 'Gás Diesel', 
    10: 'Nova Tecnologia', 11: 'Célula de Combustível', 12: 'Diesel Elétrico', -1: 'Ausente'
}
dict_posicao_volante = { 1: 'Volante à Direita (RHD)', 2: 'Volante à Esquerda (LHD)', -1: 'Ausente', 9: 'Desc.'}

# 3. Pessoa
dict_sexo = {1: 'Masculino', 2: 'Feminino', 3: 'Desconhecido', -1: 'Ausente'}
dict_faixa_etaria = {
    1: '0-5', 2: '6-10', 3: '11-15', 4: '16-20', 5: '21-25', 6: '26-35', 
    7: '36-45', 8: '46-55', 9: '56-65', 10: '66-75', 11: '75+', -1: 'Desconhecido'
}
dict_classe_vitima = {
    1: 'Motorista/Piloto', 
    2: 'Passageiro', 
    3: 'Pedestre'
}
dict_tipo_vitima = dict_tipo_veiculo.copy()
dict_tipo_vitima[0] = 'Pedestre'

dict_movimento_pedestre = {
    0: 'Não é pedestre', 1: 'Atravessando (perto, em vista)', 2: 'Atravessando (perto, oculto)', 
    3: 'Atravessando (longe, em vista)', 4: 'Atravessando (longe, oculto)', 
    5: 'Na pista (parado)', 6: 'Na pista (parado, oculto)', 
    7: 'Andando (a favor do tráfego)', 8: 'Andando (contra o tráfego)', 9: 'Desc.', -1: 'Ausente'
}
dict_passageiro_carro = {0: 'Não é passageiro', 1: 'Banco da Frente', 2: 'Banco de Trás', 9: 'Desc.', -1: 'Ausente'}
dict_passageiro_onibus = {0: 'Não é passageiro', 1: 'Embarcando', 2: 'Desembarcando', 3: 'Em pé', 4: 'Sentado', 9: 'Desc.', -1: 'Ausente'}

### Creation of Dimensions

In [ ]:
def create_dim(df_source, col_origem, dict_labels, nome_dim):
    df_dim = df_source.select(F.col(col_origem).alias(f"ide_{nome_dim}")).distinct()
    mapping_expr = F.create_map([F.lit(x) for x in sum(dict_labels.items(), ())])
    df_dim = df_dim.withColumn(f"dec_{nome_dim}", mapping_expr[F.col(f"ide_{nome_dim}")])
    df_dim = df_dim.withColumn(
        f"dec_{nome_dim}", 
        F.coalesce(F.col(f"dec_{nome_dim}"), F.lit("DESCRIÇÃO NÃO ENCONTRADA"))
    )
    window_spec = Window.orderBy(f"ide_{nome_dim}")
    df_dim = df_dim.withColumn(f"suk_{nome_dim}", F.row_number().over(window_spec))
    return df_dim.select(f"suk_{nome_dim}", f"ide_{nome_dim}", f"dec_{nome_dim}")

dim_severidade = create_dim(df_silver, "accident_severity", dict_severidade, "severidade")
dim_clima = create_dim(df_silver, "weather_conditions", dict_clima, "clima")
dim_pista = create_dim(df_silver, "road_surface_conditions", dict_cod_pista, "pista")
dim_luz = create_dim(df_silver, "light_conditions", dict_luz, "luz")
dim_urbano = create_dim(df_silver, "urban_or_rural_area", dict_urbano_rural, "urbano")
dim_sexo = create_dim(df_silver, "sex_of_driver", dict_sexo, "sexo")
dim_tipo_veiculo = create_dim(df_silver, "vehicle_type", dict_tipo_veiculo, "tipo_veiculo")
dim_tipo_via = create_dim(df_silver, "road_type", dict_tipo_via, "tipo_via")
dim_cruzamento = create_dim(df_silver, "junction_detail", dict_cruzamento_detalhe, "cruzamento")
dim_controle_cruz = create_dim(df_silver, "junction_control", dict_cruzamento_controle, "controle_cruz")
dim_ped_fisico = create_dim(df_silver, "pedestrian_crossing_physical_facilities", dict_ped_fisico, "ped_fisico")
dim_cod_esp = create_dim(df_silver, "special_conditions_at_site", dict_cod_especiais, "cod_esp")
dim_perigos = create_dim(df_silver, "carriageway_hazards", dict_perigos_pista, "perigos")
dim_manobra = create_dim(df_silver, "vehicle_manoeuvre", dict_manobra, "manobra")
dim_local_veic = create_dim(df_silver, "vehicle_location_restricted_lane", dict_local_veiculo, "local_veic")
dim_propulsao = create_dim(df_silver, "propulsion_code", dict_propulsao, "propulsao")
dim_volante = create_dim(df_silver, "was_vehicle_left_hand_drive", dict_posicao_volante, "volante")
dim_faixa_etaria = create_dim(df_silver, "age_band_of_driver", dict_faixa_etaria, "faixa_etaria")
dim_classe_vit = create_dim(
    df_silver.withColumn("casualty_class", F.col("casualty_class").cast("int")), 
    "casualty_class", 
    dict_classe_vitima, 
    "classe_vit"
)
dim_tipo_vit = create_dim(df_silver, "casualty_type", dict_tipo_vitima, "tipo_vit")
dim_mov_ped = create_dim(df_silver, "pedestrian_movement", dict_movimento_pedestre, "mov_ped")
dim_pass_carro = create_dim(df_silver, "car_passenger", dict_passageiro_carro, "pass_carro")
dim_pass_bus = create_dim(df_silver, "bus_or_coach_passenger", dict_passageiro_onibus, "pass_bus")


dim_data = df_silver.select(F.col("accident_timestamp").alias("data_completa")).distinct() \
    .withColumn("suk_data", F.date_format("data_completa", "yyyyMMdd").cast("int")) \
    .withColumn("val_ano", F.year("data_completa")) \
    .withColumn("val_mes", F.month("data_completa")) \
    .withColumn("val_dia_mes", F.dayofmonth("data_completa")) \
    .withColumn("dec_dia_semana", F.date_format("data_completa", "EEEE")) \
    .withColumn("flag_fim_de_semana", F.when(F.dayofweek("data_completa").isin([1, 7]), "S").otherwise("N")) \
    .select("suk_data", F.col("suk_data").alias("ide_data"), "data_completa", "val_ano", "val_mes", "val_dia_mes", "dec_dia_semana", "flag_fim_de_semana")

dim_tempo = df_silver.select(F.col("accident_time").alias("hora_completa")).distinct() \
    .withColumn("ide_tempo", F.regexp_replace("hora_completa", ":", "").cast("int")) \
    .withColumn("val_hora_dia", F.split("hora_completa", ":")[0].cast("int")) \
    .withColumn("dec_periodo_dia", 
                F.when(F.col("val_hora_dia") < 12, "Manhã")
                 .when(F.col("val_hora_dia") < 18, "Tarde")
                 .otherwise("Noite")) \
    .withColumn("suk_tempo", F.row_number().over(Window.orderBy("hora_completa"))) \
    .select("suk_tempo", "ide_tempo", "hora_completa", "val_hora_dia", "dec_periodo_dia")


### Creation Facts

In [ ]:
fat_acidente = df_silver.alias("orig") \
    .join(dim_data, F.col("orig.accident_timestamp") == dim_data.data_completa, "left") \
    .join(dim_tempo, F.col("orig.accident_time") == dim_tempo.hora_completa, "left") \
    .join(dim_severidade, F.col("orig.accident_severity") == dim_severidade.ide_severidade, "left") \
    .join(dim_urbano, F.col("orig.urban_or_rural_area") == dim_urbano.ide_urbano, "left") \
    .join(dim_clima, F.col("orig.weather_conditions") == dim_clima.ide_clima, "left") \
    .join(dim_pista, F.col("orig.road_surface_conditions") == dim_pista.ide_pista, "left") \
    .join(dim_luz, F.col("orig.light_conditions") == dim_luz.ide_luz, "left") \
    .join(dim_tipo_via, F.col("orig.road_type") == dim_tipo_via.ide_tipo_via, "left") \
    .join(dim_cruzamento, F.col("orig.junction_detail") == dim_cruzamento.ide_cruzamento, "left") \
    .join(dim_controle_cruz, F.col("orig.junction_control") == dim_controle_cruz.ide_controle_cruz, "left") \
    .join(dim_ped_fisico, F.col("orig.pedestrian_crossing_physical_facilities") == dim_ped_fisico.ide_ped_fisico, "left") \
    .join(dim_cod_esp, F.col("orig.special_conditions_at_site") == dim_cod_esp.ide_cod_esp, "left") \
    .join(dim_perigos, F.col("orig.carriageway_hazards") == dim_perigos.ide_perigos, "left")

fat_acidente = fat_acidente.select(
    F.col("orig.accident_index").alias("ide_acidente"),
    F.col("orig.number_of_vehicles").alias("val_qtd_veiculos"),
    F.col("orig.number_of_casualties").alias("val_qtd_vitimas"),
    F.col("orig.longitude").alias("val_longitude"),
    F.col("orig.latitude").alias("val_latitude"),
    F.col("orig.speed_limit").alias("val_limite_velocidade"),
    F.col("suk_data").alias("fok_data"),
    F.col("suk_tempo").alias("fok_tempo"),
    F.col("suk_severidade").alias("fok_severidade"),
    F.col("suk_urbano").alias("fok_urbano"),
    F.col("suk_clima").alias("fok_clima"),
    F.col("suk_pista").alias("fok_pista"),
    F.col("suk_luz").alias("fok_luz"),
    F.col("suk_tipo_via").alias("fok_tipo_via"),
    F.col("suk_cruzamento").alias("fok_cruzamento"),
    F.col("suk_controle_cruz").alias("fok_controle_cruz"),
    F.col("suk_ped_fisico").alias("fok_ped_fisico"),
    F.col("suk_cod_esp").alias("fok_cod_esp"),
    F.col("suk_perigos").alias("fok_perigos")
).distinct()

fat_veiculo = df_silver.alias("orig") \
    .join(dim_tipo_veiculo, F.col("orig.vehicle_type") == dim_tipo_veiculo.ide_tipo_veiculo, "left") \
    .join(dim_manobra, F.col("orig.vehicle_manoeuvre") == dim_manobra.ide_manobra, "left") \
    .join(dim_sexo, F.col("orig.sex_of_driver") == dim_sexo.ide_sexo, "left") \
    .join(dim_faixa_etaria, F.col("orig.age_band_of_driver") == dim_faixa_etaria.ide_faixa_etaria, "left") \
    .join(dim_propulsao, F.col("orig.propulsion_code") == dim_propulsao.ide_propulsao, "left") \
    .join(dim_volante, F.col("orig.was_vehicle_left_hand_drive") == dim_volante.ide_volante, "left")

fat_veiculo = fat_veiculo.select(
    F.col("orig.accident_index").alias("ide_acidente"),
    F.col("orig.vehicle_reference").alias("ide_veiculo"),
    F.col("orig.age_of_driver").alias("val_idade_motorista"),
    F.col("orig.age_of_vehicle").alias("val_idade_veiculo"),
    F.col("suk_tipo_veiculo").alias("fok_tipo_veiculo"),
    F.col("suk_manobra").alias("fok_manobra"),
    F.col("suk_sexo").alias("fok_sexo_motorista"),
    F.col("suk_faixa_etaria").alias("fok_faixa_etaria_mot"),
    F.col("suk_propulsao").alias("fok_propulsao"),
    F.col("suk_volante").alias("fok_volante")
).distinct()

fat_vitima = df_silver.alias("orig") \
    .join(dim_classe_vit, F.col("orig.casualty_class") == dim_classe_vit.ide_classe_vit, "left") \
    .join(dim_severidade, F.col("orig.casualty_severity") == dim_severidade.ide_severidade, "left") \
    .join(dim_sexo, F.col("orig.sex_of_casualty") == dim_sexo.ide_sexo, "left") \
    .join(dim_faixa_etaria, F.col("orig.age_band_of_casualty") == dim_faixa_etaria.ide_faixa_etaria, "left") \
    .join(dim_mov_ped, F.col("orig.pedestrian_movement") == dim_mov_ped.ide_mov_ped, "left")

fat_vitima = fat_vitima.select(
    F.col("orig.accident_index").alias("ide_acidente"),
    F.col("orig.vehicle_reference").alias("ide_veiculo"),
    F.col("orig.casualty_reference").alias("ide_vitima"),
    F.col("orig.age_of_casualty").alias("val_idade_vitima"),
    F.col("suk_classe_vit").alias("fok_classe_vitima"),
    F.col("suk_severidade").alias("fok_severidade_vitima"),
    F.col("suk_sexo").alias("fok_sexo_vitima"),
    F.col("suk_faixa_etaria").alias("fok_faixa_etaria_vit"),
    F.col("suk_mov_ped").alias("fok_mov_pedestre")
).distinct()

### Load on Database

In [ ]:
db_url = "jdbc:postgresql://localhost:5432/gis"
db_props = {"user": "postgres", "password": "postgres", "driver": "org.postgresql.Driver"}

tables_to_truncate = ["dw_gold.dim_data", "dw_gold.fat_vitima"] 

tables_to_clean = [
    "dw_gold.fat_vitima", "dw_gold.fat_veiculo", "dw_gold.fat_acidente",
    "dw_gold.dim_data", "dw_gold.dim_tempo", "dw_gold.dim_severidade", 
    "dw_gold.dim_urbano", "dw_gold.dim_clima", "dw_gold.dim_pista", 
    "dw_gold.dim_luz", "dw_gold.dim_tipo_via", "dw_gold.dim_cruzamento",
    "dw_gold.dim_controle_cruz", "dw_gold.dim_ped_fisico", "dw_gold.dim_cod_esp",
    "dw_gold.dim_perigos", "dw_gold.dim_tipo_veiculo", "dw_gold.dim_manobra",
    "dw_gold.dim_local_veic", "dw_gold.dim_propulsao", "dw_gold.dim_volante",
    "dw_gold.dim_sexo", "dw_gold.dim_faixa_etaria", "dw_gold.dim_classe_vit",
    "dw_gold.dim_tipo_vit", "dw_gold.dim_mov_ped", "dw_gold.dim_pass_carro",
    "dw_gold.dim_pass_bus"
]

try:
    connection = spark._sc._gateway.jvm.java.sql.DriverManager.getConnection(
        db_url, "postgres", "postgres"
    )
    statement = connection.createStatement()
    sql_clean = """
    TRUNCATE TABLE 
        dw_gold.fat_vitima, dw_gold.fat_veiculo, dw_gold.fat_acidente,
        dw_gold.dim_data, dw_gold.dim_tempo, dw_gold.dim_severidade, 
        dw_gold.dim_urbano, dw_gold.dim_clima, dw_gold.dim_pista, 
        dw_gold.dim_luz, dw_gold.dim_tipo_via, dw_gold.dim_cruzamento,
        dw_gold.dim_controle_cruz, dw_gold.dim_ped_fisico, dw_gold.dim_cod_esp,
        dw_gold.dim_perigos, dw_gold.dim_tipo_veiculo, dw_gold.dim_manobra,
        dw_gold.dim_local_veic, dw_gold.dim_propulsao, dw_gold.dim_volante,
        dw_gold.dim_sexo, dw_gold.dim_faixa_etaria, dw_gold.dim_classe_vit,
        dw_gold.dim_tipo_vit, dw_gold.dim_mov_ped, dw_gold.dim_pass_carro,
        dw_gold.dim_pass_bus
    CASCADE;
    """
    statement.execute(sql_clean)
    connection.close()
    print("Tabelas limpas com sucesso.") 

except Exception as e:
    print(f"Erro na limpeza: {e}")


dimensoes_para_salvar = [
    (dim_data, "dw_gold.dim_data"), (dim_tempo, "dw_gold.dim_tempo"), 
    (dim_severidade, "dw_gold.dim_severidade"), (dim_urbano, "dw_gold.dim_urbano"),
    (dim_clima, "dw_gold.dim_clima"), (dim_pista, "dw_gold.dim_pista"), 
    (dim_luz, "dw_gold.dim_luz"), (dim_tipo_via, "dw_gold.dim_tipo_via"), 
    (dim_cruzamento, "dw_gold.dim_cruzamento"), (dim_controle_cruz, "dw_gold.dim_controle_cruz"),
    (dim_ped_fisico, "dw_gold.dim_ped_fisico"), (dim_cod_esp, "dw_gold.dim_cod_esp"),
    (dim_perigos, "dw_gold.dim_perigos"), (dim_tipo_veiculo, "dw_gold.dim_tipo_veiculo"), 
    (dim_manobra, "dw_gold.dim_manobra"), (dim_local_veic, "dw_gold.dim_local_veic"),
    (dim_propulsao, "dw_gold.dim_propulsao"), (dim_volante, "dw_gold.dim_volante"),
    (dim_sexo, "dw_gold.dim_sexo"), (dim_faixa_etaria, "dw_gold.dim_faixa_etaria"),
    (dim_classe_vit, "dw_gold.dim_classe_vit"), (dim_tipo_vit, "dw_gold.dim_tipo_vit"),
    (dim_mov_ped, "dw_gold.dim_mov_ped"), (dim_pass_carro, "dw_gold.dim_pass_carro"),
    (dim_pass_bus, "dw_gold.dim_pass_bus")
]

print(f"--- Salvando {len(dimensoes_para_salvar)} Tabelas de Dimensão (Append) ---")

for df, table_name in dimensoes_para_salvar:
    print(f"  -> Salvando: {table_name} ...")
    df.write.jdbc(url=db_url, table=table_name, mode="append", properties=db_props)


fatos_para_salvar = [
    (fat_acidente, "dw_gold.fat_acidente"),
    (fat_veiculo, "dw_gold.fat_veiculo"),
    (fat_vitima, "dw_gold.fat_vitima")
]

print(f"\n--- Salvando {len(fatos_para_salvar)} Tabelas Fato (Append) ---")

for df, table_name in fatos_para_salvar:
    df.write.jdbc(url=db_url, table=table_name, mode="append", properties=db_props)

df_silver.unpersist()

Tabelas limpas com sucesso.
--- Salvando 25 Tabelas de Dimensão (Append) ---
  -> Salvando: dw_gold.dim_data ...
  -> Salvando: dw_gold.dim_tempo ...
  -> Salvando: dw_gold.dim_severidade ...
  -> Salvando: dw_gold.dim_urbano ...
  -> Salvando: dw_gold.dim_clima ...
  -> Salvando: dw_gold.dim_pista ...
  -> Salvando: dw_gold.dim_luz ...
  -> Salvando: dw_gold.dim_tipo_via ...
  -> Salvando: dw_gold.dim_cruzamento ...
  -> Salvando: dw_gold.dim_controle_cruz ...
  -> Salvando: dw_gold.dim_ped_fisico ...
  -> Salvando: dw_gold.dim_cond_esp ...
  -> Salvando: dw_gold.dim_perigos ...
  -> Salvando: dw_gold.dim_tipo_veiculo ...
  -> Salvando: dw_gold.dim_manobra ...
  -> Salvando: dw_gold.dim_local_veic ...
  -> Salvando: dw_gold.dim_propulsao ...
  -> Salvando: dw_gold.dim_volante ...
  -> Salvando: dw_gold.dim_sexo ...
  -> Salvando: dw_gold.dim_faixa_etaria ...
  -> Salvando: dw_gold.dim_classe_vit ...
  -> Salvando: dw_gold.dim_tipo_vit ...
  -> Salvando: dw_gold.dim_mov_ped ...
  -> Sa

DataFrame[accident_index: string, vehicle_reference: int, longitude: double, latitude: double, accident_severity: int, number_of_vehicles: int, number_of_casualties: int, accident_timestamp: date, day_of_week: int, accident_time: string, road_type: int, speed_limit: int, junction_detail: int, junction_control: int, pedestrian_crossing_physical_facilities: int, light_conditions: int, weather_conditions: int, road_surface_conditions: int, special_conditions_at_site: int, carriageway_hazards: int, urban_or_rural_area: int, vehicle_type: int, vehicle_manoeuvre: int, vehicle_location_restricted_lane: int, was_vehicle_left_hand_drive: int, sex_of_driver: int, age_of_driver: int, age_band_of_driver: int, propulsion_code: int, age_of_vehicle: int, casualty_reference: int, casualty_class: int, sex_of_casualty: int, age_of_casualty: int, age_band_of_casualty: int, casualty_severity: int, pedestrian_movement: int, car_passenger: int, bus_or_coach_passenger: int, casualty_type: int]